## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2026-02-24-23-00-00 +0000,wsj,The Trump administration is weighing a possibl...,https://www.wsj.com/politics/policy/trump-admi...
1,2026-02-24-22-58-46 +0000,bbc,US threatens Anthropic with deadline in disput...,https://www.bbc.com/news/articles/cjrq1vwe73po...
2,2026-02-24-22-55-00 +0000,wsj,AI-Induced Tech Selloff Spoils the IPO Parade,https://www.wsj.com/finance/stocks/ai-induced-...
3,2026-02-24-22-53-57 +0000,nyt,House Narrowly Rejects Air Safety Bill After P...,https://www.nytimes.com/2026/02/24/us/house-ro...
4,2026-02-24-22-51-26 +0000,nyt,Guest List for State of the Union Includes Eps...,https://www.nytimes.com/2026/02/24/us/politics...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2434/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
0,trump,64
53,new,32
36,union,26
254,war,24
500,mexico,20


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
371,2026-02-24-02-00-00 +0000,wsj,President Trump will use his State of the Unio...,https://www.wsj.com/politics/policy/trump-to-s...,191
287,2026-02-24-11-54-00 +0000,wsj,"A bit awkward? Supreme Court justices, some of...",https://www.wsj.com/politics/trump-to-face-off...,170
319,2026-02-24-10-30-00 +0000,wsj,Tuesday’s State of the Union address gives Pre...,https://www.wsj.com/economy/sizing-up-the-u-s-...,156
199,2026-02-24-16-59-59 +0000,nypost,Trump weighs new batch of national security ta...,https://nypost.com/2026/02/24/business/trump-w...,140
273,2026-02-24-13-00-00 +0000,wsj,Dozens of congressional Democrats are preparin...,https://www.wsj.com/politics/policy/democrats-...,135


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
371,191,2026-02-24-02-00-00 +0000,wsj,President Trump will use his State of the Unio...,https://www.wsj.com/politics/policy/trump-to-s...
408,84,2026-02-23-23-09-36 +0000,nypost,El Mencho killing marks new war against cartel...,https://nypost.com/2026/02/23/world-news/el-me...
287,61,2026-02-24-11-54-00 +0000,wsj,"A bit awkward? Supreme Court justices, some of...",https://www.wsj.com/politics/trump-to-face-off...
116,57,2026-02-24-20-13-35 +0000,bbc,Mandelson's lawyers say he was arrested over '...,https://www.bbc.com/news/articles/c05vey47yddo...
381,56,2026-02-24-01-03-34 +0000,nypost,Mexico beach named best in the world — right b...,https://nypost.com/2026/02/23/world-news/mexic...
99,50,2026-02-24-20-50-39 +0000,nypost,Defiant Zelensky refuses surrendering land on ...,https://nypost.com/2026/02/24/world-news/defia...
197,49,2026-02-24-17-00-00 +0000,nypost,OpenAI asks California watchdog to probe who’s...,https://nypost.com/2026/02/24/business/openai-...
109,47,2026-02-24-20-26-51 +0000,nypost,"Ozempic, Wegovy prices to drop up to 50% as No...",https://nypost.com/2026/02/24/business/ozempic...
351,44,2026-02-24-04-21-44 +0000,nyt,"Peter Mandelson, Ex-Ambassador to the U.S., Is...",https://www.nytimes.com/2026/02/23/world/europ...
354,36,2026-02-24-04-00-00 +0000,wsj,"As War Enters Fifth Year, Ukraine Shows Russia...",https://www.wsj.com/world/as-war-enters-fifth-...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
